In [ ]:
# coded by Sayan Chandra CS20M057
# CS6910_DEEP LEARNING : (prof.) Mitesh M. Khapra

In [1]:
!pip install wandb

     |████████████████████████████████| 2.0MB 11.8MB/s 
     |████████████████████████████████| 133kB 34.0MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
     |████████████████████████████████| 163kB 50.6MB/s 
     |████████████████████████████████| 71kB 4.8MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=efc7b3de9935dc3ba022662c66bea6c98195a7f7748460bf3efc37086c30da52
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=a0161815b354265d76b52cd38a1312dc413060c43597520c27ffe4807617572b
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [ ]:
!wandb login 

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
import wandb
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist, mnist
import numpy as np
from copy import deepcopy as clone
from sklearn.utils import shuffle as jumble


#from sklearn.datasets import make_blobs
#data, lables = make_blobs(n_samples=10000, centers=2, n_features=5, random_state=0)

In [10]:

sweep_config={
    'method' : 'random',
    'metric' : {
        'name' : 'Training Accuracy',
        'goal' : 'maximize',
    },
    'parameters' : {
        'epochs' : {
            'values' : [2, 7, 10],
        },
        'batch_size' : {
            'values' : [32, 64, 128],
        },
        'layer_size' : {
            'values' : [[64], [32], [16, 32], [128], [32, 64]],
        },
        'learning_rate' : {
            'values' : [0.1, 1, 3, 3.3, 4],
        },
        'optimizer' : {
            'values' : ['GD', 'MBGD', 'NAGD', 'SGD', 'RMSPROP', 'ADAM', 'NADAM'],
        },
        'activations' : {
            'values' : ['sigmoid', 'tanh', 'reLU'],
        },
        'initialisations' : {
            'values' : ['random', 'xavier'],
        },
        'lossfunc' : {
            'values' : ['crossentropy'], #, 'squarederror'],
        },
        'regilarizercoeff' : {
            'values' : [0.00005, 0.0001, 0, 0.0003]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, entity="blackcloud", project="cs6910_dl_assignment_1")

Create sweep with ID: 1hsulpuf
Sweep URL: https://wandb.ai/blackcloud/cs6910_dl_assignment_1/sweeps/1hsulpuf


In [11]:
# load dataset
def loadData():
    (trainX, trainy), (testX, testy) = fashion_mnist.load_data()
    N_inputs, dim_x_inp, dim_y_inp=len(trainX), len(trainX[0]), len(trainX[0][0])
    inputs=np.zeros((N_inputs, dim_x_inp*dim_y_inp))
    sum1=np.zeros((dim_x_inp*dim_y_inp))
    for i in range(N_inputs):
       inputs[i]=trainX[i].flatten()
       sum1+=inputs[i]
    mean1=sum1/N_inputs
    dim_inp=inputs.shape[1]

    sum2=np.zeros((dim_x_inp*dim_y_inp))
    testx=np.zeros((len(testX), dim_x_inp*dim_y_inp))
    for i in range(len(testX)):
      testx[i]=testX[i].flatten()
      sum2+=testx[i]
    mean2=sum2/len(testX)

    for i in range(N_inputs):
      inputs[i]-=mean1
      inputs[i]/=255
    trainx=inputs.astype('float64')

    for i in range(len(testX)):
      testx[i]-=mean2
      testx[i]/=255
    testx=testx.astype('float64')
    trainx, trainy = jumble(trainx, trainy)
    testx, testy = jumble(testx, testy)
    return (trainx[6000: ], trainy[6000:], trainx[:6000], trainy[:6000], testx, testy)


In [ ]:
trainx, trainy, valx, valy, testx, testy=loadData()
dim_inp=trainx.shape[1]
lable_names=["t-shirt/top", 'trouser/pants', 'pullover shirt', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
wandb.init(project="cs6910_dl_assignment_1")

In [13]:


## main variables inputs matrix, N_inputs, dim_inp

class MLSN (object): ## Multi layer Sigmoid Neurons
  def __init__(self, szInp, hiddenL, numOp, actvfunc, initfunc, lossfunc, lmda, vx, vy, xtest, ytest) :## size of inp, num of hidden layers, num of neurons per layer, num of output.
    self.szInp=szInp
    self.hiddenL=hiddenL
    self.numOp=numOp
    self.actvfunc=actvfunc
    self.initfunc=initfunc
    self.lossfunc=lossfunc
    self.lmda=lmda
    self.vx=vx
    self.vy=vy
    self.xtest=xtest
    self.ytest=ytest
    
    self.genericLayer=[szInp]+hiddenL+[numOp]
    print(self.genericLayer)

    self.lastpreactv=[]  

    self.weights = []

    if self.initfunc=="random":
      for i in range(len(self.genericLayer) - 1):
        w = np.random.random((self.genericLayer[i], self.genericLayer[i + 1]))
        (self.weights).append(w)
    elif self.initfunc=="xavier":
      for i in range(len(self.genericLayer) - 1):
        w = np.random.normal(0, 1/np.sqrt(self.genericLayer[i]), (self.genericLayer[i], self.genericLayer[i + 1]))
        (self.weights).append(w)

    

    self.biases=[]
    for i in range(len(self.genericLayer) - 1):
      b = np.random.normal(0, 1, self.genericLayer[i + 1])
      (self.biases).append(b)


    self.prevWeights=[np.zeros(xx.shape) for xx in self.weights]
    self.prevBiases=[np.zeros(xx.shape) for xx in self.biases]


    self.activations = []
    for i in range(len(self.genericLayer)):
      a = np.zeros(self.genericLayer[i])
      (self.activations).append(a)


    self.delLdelA=clone(self.activations)
    self.delLdelH=clone(self.activations)
    

    self.delLdelW=clone(self.prevWeights)
    self.delLdelB=clone(self.prevBiases)

  def reinitdels(self):
    self.delLdelW=[np.zeros(xx.shape) for xx in self.weights]
    self.delLdelB=[np.zeros(xx.shape) for xx in self.biases]
    

  def calculate_a_i(self, i, w):
    return np.dot(i,w)

  sigmoid = lambda self, pa: np.array([(np.exp(term) if term<0 else 1)/(1+np.exp(term if term<0 else -term)) for term in pa])
  tanh = lambda self, pa: (np.exp(pa-np.max(pa))-np.exp(-pa-np.max(pa)))/(np.exp(pa-np.max(pa))+np.exp(-pa-np.max(pa)))
  reLU = lambda self, pa: np.maximum(0,pa*0.001)

  def calculate_h_i(self,pa):
    if self.actvfunc=="sigmoid" : return self.sigmoid(pa) 
    elif self.actvfunc=="tanh" :  return np.tanh(pa)
    elif self.actvfunc=="reLU" : return self.reLU(pa)+1e-8
    #return 1/(1+np.exp(-pa))

  def softmax(self,outpa):
    mx=np.max(outpa)
    outpa=outpa-mx
    return np.exp(outpa)/np.sum(np.exp(outpa)+1e-9)


  def feedFrwd(self,curinp,ws,bs):
    self.activations[0]=curinp
    for i in range(len(ws)-1):
      preActv=self.calculate_a_i(curinp, ws[i])+bs[i]
      actv=self.calculate_h_i(preActv)
      self.activations[i+1]=actv
      curinp=actv
    outPreActv=np.dot(curinp, ws[-1])+bs[-1]
    self.lastpreactv=outPreActv
    outActv=self.softmax(outPreActv)
    self.activations[-1]=outActv

  def derivativeActv(self, i):
    if self.actvfunc=="sigmoid" : return self.activations[i]*(1-self.activations[i])
    elif self.actvfunc=="tanh" : return 1-np.tanh(self.activations[i])**2
    elif self.actvfunc=="reLU" : 
      arrr= clone(self.activations[i])
      arrr[arrr>0]=1; arrr[arrr<=0]=0
      return arrr

  def crossentropy(self, curidx, ys):
    a=np.zeros(self.numOp)
    a[ys[curidx]]=1
    a=-(a-self.activations[-1])
    self.delLdelA[-1]=a

  def squarederror(self, curidx, ys):
    predclass=ys[curidx]
    pp=np.exp(clone(self.lastpreactv))
    self.lastpreactv=np.exp(self.lastpreactv)
    pp[predclass]=0
    add1, add2, add3=np.sum(pp), np.sum(pp**2), np.sum(self.lastpreactv)
    xyz = ((add3 * (self.lastpreactv + add1) - (add2 + add1**2) )/(add3**3))*pp
    xyz[predclass] = ((-add2 - add1**2)/(add3**3)) *(self.lastpreactv[predclass])
    self.delLdelA[-1]=xyz


    '''
    ret=[]
    ll=len(self.activations[-1])
    for j in range(ll):
      add=0
      for i in range(ll):
        add+=2*(self.activations[-1][i]-ys[curidx])*self.activations[-1][i]*((i==j)-self.activations[-1][j])
      ret.append(add)
    self.delLdelA[-1]=np.array(ret)/len(ys)
    '''



  def lossDerivative(self, curidx,  ys):
    if self.lossfunc=="crossentropy" : self.crossentropy(curidx, ys)
    elif self.lossfunc=="squarederror" : self.squarederror(curidx, ys)

  def backProp(self, curidx, ys):
    self.lossDerivative(curidx, ys)
    for i in range(len(self.genericLayer)-2,-1,-1):
      sz1=len(self.delLdelA[i+1])
      sz2=len(self.activations[i])
      self.delLdelW[i]+=np.matmul(self.delLdelA[i+1].reshape(sz1,1),self.activations[i].reshape(sz2,1).T).T + self.lmda*self.weights[i]
      self.delLdelH[i]=np.dot(self.weights[i], self.delLdelA[i+1])
      self.delLdelA[i]=np.multiply(self.delLdelH[i], self.derivativeActv(i))
      self.delLdelB[i]+=self.delLdelA[i+1]


  def TrainAccuracy(self,data,lables):
    count=0
    curloss=0
    for i in range(len(data)):#
      curinp=data[i]#
      self.feedFrwd(curinp, self.weights, self.biases)
      count+=(np.argmax(self.activations[-1])==lables[i])
      if self.lossfunc=="crossentropy":
        curloss+=-np.log(max(self.activations[-1]))+(self.lmda/2)*np.sum([np.sum(w**2) for w in self.weights])
      if self.lossfunc=="squarederror":
        dummy=np.zeros(len(self.activations[-1]))
        dummy[lables[i]]=1
        curloss+=np.sum((self.activations[-1]-dummy)**2)
    return (count/len(data), curloss/len(data))

  def TestAccuracy(self,data,lables):
    count=0
    curloss=0
    predictions=[]
    matrix=np.zeros((10,10))
    for i in range(len(data)):#
      curinp=data[i]#
      self.feedFrwd(curinp, self.weights, self.biases)
      predictions.append(np.argmax(self.activations[-1]))
      xxxxx, yyyyy=np.argmax(self.activations[-1]), lables[i]
      count+=(xxxxx==yyyyy)
      matrix[xxxxx][yyyyy]+=1
      curloss+=-np.log(max(self.activations[-1]))+(self.lmda/2)*np.sum([np.sum(w**2) for w in self.weights])

    wandb.log({"confusion matrix": wandb.plot.confusion_matrix(probs=None,
                                                              y_true=lables,
                                                              preds=predictions,
                                                              class_names=lable_names)})
    wandb.log({"confusion matrix v2.0": wandb.plots.HeatMap(lable_names, lable_names, matrix, show_text=True)})
    return (count/len(data), curloss/len(data))


  def AccuracyLossLogging(self, epch, xs, ys):
      a,b=self.TrainAccuracy(xs, ys)
      c,d=self.TrainAccuracy(self.vx, self.vy)
      #print(a,c,b,d)
      wandb.log({"Epochs":epch, "Training Accuracy":a, "Training loss":b, "Val Accuracy":c, "Val Loss":d})
      return -1


  def trainNADAM(self, xs, ys, run, batchsize, rate):
    beta1, beta2, eps=0.9, 0.9, 1e-7
    m_w=[np.zeros(xx.shape) for xx in self.delLdelW]
    m_b=[np.zeros(xx.shape) for xx in self.delLdelB]
    v_w=[np.zeros(xx.shape) for xx in self.delLdelW]
    v_b=[np.zeros(xx.shape) for xx in self.delLdelB]
    var=0
    for _ in range(run):
      var+=batchsize
      self.reinitdels()
      
      for i in range(len(xs)):
        curinp=xs[i]
        self.feedFrwd(curinp,self.weights, self.biases)
        self.backProp(i, ys)
        if i%(batchsize-1)==0 and i!=0:
          var+=1
          for j in range(len(self.weights)):

            m_w[j]=m_w[j]*beta1 + self.delLdelW[j]*(1-beta1)
            m_b[j]=m_b[j]*beta1 + self.delLdelB[j]*(1-beta1)

            v_w[j]=v_w[j]*beta2 + (self.delLdelW[j]**2)*(1-beta2)
            v_b[j]=v_b[j]*beta2 + (self.delLdelB[j]**2)*(1-beta2)
            
            if var>0:
              m_w[j]/=(1-np.power(beta1,var))
              m_b[j]/=(1-np.power(beta1,var))
              v_w[j]/=(1-np.power(beta2,var))+1e-7
              v_b[j]/=(1-np.power(beta2,var))+1e-7

            m_w[j]=m_w[j]*beta1 + self.delLdelW[j]*(1-beta1)
            m_b[j]=m_b[j]*beta1 + self.delLdelB[j]*(1-beta1)
            v_w[j]=v_w[j]*beta2 + (self.delLdelW[j]**2)*(1-beta2)
            v_b[j]=v_b[j]*beta2 + (self.delLdelB[j]**2)*(1-beta2)

            self.weights[j]-=m_w[j]*((rate/batchsize)/np.sqrt(v_w[j]+eps))
            self.biases[j]-=m_b[j]*((rate/batchsize)/np.sqrt(v_b[j]+eps))


          self.reinitdels()

      self.AccuracyLossLogging(_, xs, ys)
    self.TestAccuracy(self.xtest, self.ytest)

  def trainADAM(self, xs, ys, run, batchsize, rate):
    beta1, beta2, eps=0.9, 0.9, 1e-8
    m_w=[np.zeros(xx.shape) for xx in self.delLdelW]
    m_b=[np.zeros(xx.shape) for xx in self.delLdelB]
    v_w=[np.zeros(xx.shape) for xx in self.delLdelW]
    v_b=[np.zeros(xx.shape) for xx in self.delLdelB]
    var=0
    for _ in range(run):
      var+=batchsize
      self.reinitdels()
      
      for i in range(len(xs)):
        curinp=xs[i]
        self.feedFrwd(curinp,self.weights, self.biases)
        self.backProp(i, ys)
        if i%(batchsize-1)==0 and i!=0:
          var+=1
          for j in range(len(self.weights)):
            m_w[j]=m_w[j]*beta1 + self.delLdelW[j]*(1-beta1)
            m_b[j]=m_b[j]*beta1 + self.delLdelB[j]*(1-beta1)

            v_w[j]=v_w[j]*beta2 + (self.delLdelW[j]**2)*(1-beta2)
            v_b[j]=v_b[j]*beta2 + (self.delLdelB[j]**2)*(1-beta2)
            
            if var:
              m_w[j]=np.divide(m_w[j],1-np.power(beta1,var))
              m_b[j]=np.divide(m_b[j],1-np.power(beta1,var))
              v_w[j]=np.divide(v_w[j],1-np.power(beta2,var))
              v_b[j]=np.divide(v_b[j],1-np.power(beta2,var))

            self.weights[j]-=((m_w[j]*(rate/batchsize))/np.sqrt(v_w[j]+eps))
            self.biases[j]-=((m_b[j]*(rate/batchsize))/np.sqrt(v_b[j]+eps))

          self.reinitdels()

      self.AccuracyLossLogging(_, xs, ys)
    self.TestAccuracy(self.xtest, self.ytest)


  def trainRMSPROP(self, xs, ys, run, batchsize, rate):
    beta, eps=0.8, 1e-8
    w_s=[np.zeros(xx.shape) for xx in self.delLdelW]
    b_s=[np.zeros(xx.shape) for xx in self.delLdelB]
    for _ in range(run):
      
      self.reinitdels()
      
      for i in range(len(xs)):
        curinp=xs[i]
        self.feedFrwd(curinp,self.weights, self.biases)
        self.backProp(i, ys)
        if i%(batchsize-1)==0 and i!=0:
          for j in range(len(self.weights)):
            w_s[j]=w_s[j]*beta + (self.delLdelW[j]**2)*(1-beta)
            b_s[j]=b_s[j]*beta + (self.delLdelB[j]**2)*(1-beta)
            self.weights[j]=self.weights[j] - self.delLdelW[j]*((rate/batchsize)/np.sqrt(w_s[j]*eps))
            self.biases[j]=self.biases[j] - self.delLdelB[j]*((rate/batchsize)/np.sqrt(b_s[j]*eps))

          self.reinitdels()

      self.AccuracyLossLogging(_, xs, ys)
    self.TestAccuracy(self.xtest, self.ytest)


  def trainNAGD(self, xs, ys, run, batchsize, rate):
    gamma=0.65
    for _ in range(run):
      self.reinitdels()
      wLA=[np.zeros(xx.shape) for xx in self.weights]
      bLA=[np.zeros(xx.shape) for xx in self.biases]
      self.prevWeights=[np.zeros(xx.shape) for xx in self.weights]
      self.prevBiases=[np.zeros(xx.shape) for xx in self.biases]

      for i in range(len(xs)):

        wLA=[self.weights[k]-self.prevWeights[k]*gamma for k in range (len(self.weights))] 
        bLA=[self.biases[k]-self.prevBiases[k]*gamma for k in range (len(self.biases))] 

        curinp=xs[i]
        self.feedFrwd(curinp, wLA, bLA)
        self.backProp(i, ys)
        if i%(batchsize-1)==0 and i!=0:
          for j in range(len(self.weights)):
            self.prevWeights[j]= self.prevWeights[j]*gamma+self.delLdelW[j]/batchsize*rate
            self.prevBiases[j]= self.prevBiases[j]*gamma+self.delLdelB[j]/batchsize*rate
            self.weights[j]-=self.prevWeights[j]
            self.biases[j]-=self.prevBiases[j]

          self.reinitdels()

      self.AccuracyLossLogging(_, xs, ys)
    self.TestAccuracy(self.xtest, self.ytest)

  def trainMBGD(self, xs, ys, run, batchsize, rate):
    gamma=0.45
    for _ in range(run):
      self.reinitdels()

      for i in range(len(xs)):
        curinp=xs[i]
        self.feedFrwd(curinp,self.weights, self.biases)
        self.backProp(i, ys)
        if i%(batchsize-1)==0 and i!=0:
          for j in range(len(self.weights)):
            www= self.prevWeights[j]*gamma+self.delLdelW[j]/batchsize*rate
            bbb= self.prevBiases[j]*gamma+self.delLdelB[j]/batchsize*rate
            self.weights[j]-=www
            self.biases[j]-=bbb
            self.prevWeights[j]=www
            self.prevBiases[j]=bbb

          self.reinitdels()

      self.AccuracyLossLogging(_, xs, ys)
    self.TestAccuracy(self.xtest, self.ytest)

  def trainGD(self, xs, ys, run, batchsize, rate):
    for _ in range(run):
      
      self.reinitdels()
      
      for i in range(len(xs)):
        curinp=xs[i]
        self.feedFrwd(curinp,self.weights, self.biases)
        self.backProp(i, ys)
        if i!=0 and (i%(batchsize-1) if batchsize!=1 else 0)==0:
          for j in range(len(self.weights)):
            self.weights[j]-=self.delLdelW[j]/batchsize*rate
            self.biases[j]-=self.delLdelB[j]/batchsize*rate

          self.reinitdels()

      self.AccuracyLossLogging(_, xs, ys)
    self.TestAccuracy(self.xtest, self.ytest)

  def trainParent(self, xs, ys, strr, run, batchsize, rate):
    if strr=="SGD" : self.trainGD(xs, ys, run, 1, rate)
    elif strr=="GD" : self.trainGD(xs, ys, run, batchsize, rate)
    elif strr=="MBGD" : self.trainMBGD(xs, ys, run, batchsize, rate)
    elif strr=="NAGD" : self.trainNAGD(xs, ys, run, batchsize, rate)
    elif strr=="RMSPROP" : self.trainRMSPROP(xs, ys, run, batchsize, rate)
    elif strr=="ADAM" : self.trainADAM(xs, ys, run, batchsize, rate)
    elif strr=="NADAM" : self.trainNADAM(xs, ys, run, batchsize, rate)


## main(){}//


#obj=MLSN(dim_inp, [32], 10, "reLU", "xavier", "crossentropy", 0.0002, valx, valy, testx, testy)
#obj.trainParent(trainx,trainy, "GD", 5, 32, 3.1)
#obj.trainParent(trainx,trainy, "MBGD", 5, 32, 3)
#obj.trainParent(trainx,trainy, "NAGD", 10, 32, 3)
#obj.trainParent(trainx,trainy, "RMSPROP", 5, 32, 3)
#obj.trainParent(trainx,trainy, "ADAM", 10, 32, 3)
#obj.trainParent(trainx,trainy, "NADAM", 10, 16, 3)
#print(obj.TestAccuracy(testx,testy))

In [14]:
def SweepParent():
  (trainx, trainy, valx, valy, testx, testy) = loadData()
  start=wandb.init()
  config=start.config
  objwandb=MLSN(trainx.shape[1], config.layer_size, 10, config.activations, config.initialisations, config.lossfunc, config.regilarizercoeff, valx, valy, testx, testy)
  objwandb.trainParent(trainx, trainy, config.optimizer, config.epochs, config.batch_size, config.learning_rate)

In [15]:
wandb.agent(sweep_id, SweepParent)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: dmn8mdqe with config:
wandb: 	activations: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 7
wandb: 	initialisations: random
wandb: 	layer_size: [128]
wandb: 	learning_rate: 0.1
wandb: 	lossfunc: crossentropy
wandb: 	optimizer: SGD
wandb: 	regilarizercoeff: 5e-05


[784, 128, 10]


wandb: Visualizing heatmap.


Epochs,6
Training Accuracy,0.77276
Training loss,3.7143
Val Accuracy,0.77517
Val Loss,3.71718
_runtime,861
_timestamp,1615641835
_step,8


Epochs,▁▂▃▅▆▇█
Training Accuracy,▆▁▃▅██▄
Training loss,▁▃▄▅▆▇█
Val Accuracy,▆▁▃▄██▅
Val Loss,▁▃▄▅▆▇█
_runtime,▁▂▃▄▆▇███
_timestamp,▁▂▃▄▆▇███
_step,▁▂▃▄▅▅▆▇█


...

Epochs,9
Training Accuracy,0.34485
Training loss,6956591.53012
Val Accuracy,0.3425
Val Loss,6956591.52993
_runtime,615
_timestamp,1615652653
_step,11


Epochs,▁▂▃▃▄▅▆▆▇█
Training Accuracy,▄▅▆▁▅▆▄█▅▁
Training loss,▁▅████▆▆▇▇
Val Accuracy,▄▅▆▁▅▆▄█▄▂
Val Loss,▁▅████▆▆▇▇
_runtime,▁▂▃▃▄▅▆▆▇███
_timestamp,▁▂▃▃▄▅▆▆▇███
_step,▁▂▂▃▄▄▅▅▆▇▇█


wandb: Agent Starting Run: 5o4wxvos with config:
wandb: 	activations: sigmoid
wandb: 	batch_size: 128
wandb: 	epochs: 10
wandb: 	initialisations: xavier
wandb: 	layer_size: [64]
wandb: 	learning_rate: 3
wandb: 	lossfunc: crossentropy
wandb: 	optimizer: NADAM
wandb: 	regilarizercoeff: 0.0001


[784, 64, 10]


wandb: Ctrl + C detected. Stopping sweep.
